In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

# Load in the training dataset with augmentation applied
train_generator = train_datagen.flow_from_directory(
    '/Users/brycemartin/Documents/AI/bird-class-data/Train',
    target_size = (224,224),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical'
)

# Only rescale the test set
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

# Load in the test dataset with only rescaling
test_generator = test_datagen.flow_from_directory(
    '/Users/brycemartin/Documents/AI/bird-class-data/Test',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
model = keras.Sequential([

    keras.layers.Conv2D(32, (3, 3), padding='same', activation=None, input_shape=(224, 224, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    # Convolutional Layer 2
    keras.layers.Conv2D(64, (3, 3), padding='same', activation=None),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    # Convolutional Layer 3
    keras.layers.Conv2D(128, (3, 3), padding='same', activation=None),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    # Flatten and classify
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),  # Dropout to prevent overfitting
    keras.layers.Dense(512, activation='relu'),  # Increase units in the dense layer
    keras.layers.Dropout(0.5),  # Dropout in the fully connected layer too
    keras.layers.Dense(200, activation='softmax')

])
model.summary()

In [ ]:
# Compile the Model
model.compile(
    optimizer = 'adam',    # Popular optimizer
    loss = 'categorical_crossentropy',   # Categorical crossentropy for classification tasks
    metrics = (['accuracy'])    # Measure the accuracy
)

In [ ]:
with tf.device('/GPU:0'):   
    # Test the model
    history =  model.fit(
        train_generator,    # Traning dataset from our generator
        validation_data = test_generator,
        epochs = 10    # See all the images 10 times
    )

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
model.evaluate(test_generator)